In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [2]:
spark = SparkSession \
        .builder \
        .master("local")\
        .appName("USAccidentAnalysis") \
        .getOrCreate()

In [3]:
df_charges = spark.read.csv("E:/Case_Study/Data/Charges_use.csv", header=True)
df_damages = spark.read.csv("E:/Case_Study/Data/Damages_use.csv", header=True)
df_endorse = spark.read.csv("E:/Case_Study/Data/Endorse_use.csv", header=True)
df_pp = spark.read.csv("E:/Case_Study/Data/Primary_Person_use.csv", header=True)
df_restrict = spark.read.csv("E:/Case_Study/Data/Restrict_use.csv", header=True)
df_Units = spark.read.csv("E:/Case_Study/Data/Units_use.csv", header=True)

In [4]:
df_charges.count()

116110

In [5]:
df_charges.head(5)

[Row(CRASH_ID='14768622', UNIT_NBR='1', PRSN_NBR='1', CHARGE='DRIVING WHILE INTOXICATED', CITATION_NBR=None),
 Row(CRASH_ID='14838637', UNIT_NBR='1', PRSN_NBR='1', CHARGE='DWI', CITATION_NBR='1600000015'),
 Row(CRASH_ID='14838641', UNIT_NBR='1', PRSN_NBR='1', CHARGE='RAN RED LIGHT SOLID (TC 544.007)', CITATION_NBR='L20440'),
 Row(CRASH_ID='14838641', UNIT_NBR='2', PRSN_NBR='1', CHARGE="NO DRIVER'S LICENSE (TC521.025)", CITATION_NBR='L23141'),
 Row(CRASH_ID='14838668', UNIT_NBR='1', PRSN_NBR='1', CHARGE='DRIVING WHILE INTOXICATED', CITATION_NBR='TX4IC50SRJD3')]

In [6]:
df_charges.show(truncate=False)

+--------+--------+--------+----------------------------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                  |CITATION_NBR|
+--------+--------+--------+----------------------------------------+------------+
|14768622|1       |1       |DRIVING WHILE INTOXICATED               |NULL        |
|14838637|1       |1       |DWI                                     |1600000015  |
|14838641|1       |1       |RAN RED LIGHT SOLID (TC 544.007)        |L20440      |
|14838641|2       |1       |NO DRIVER'S LICENSE (TC521.025)         |L23141      |
|14838668|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC50SRJD3|
|14838669|2       |1       |DWI W/BAC >.015                         |2015-000006 |
|14838670|1       |1       |DRIVING WHILE INTOXICATED               |2016-000003 |
|14838685|1       |1       |FAILED TO DRIVE SINGLE LANE, NOL, NO INS|138434825   |
|14838693|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC60UKQND|
|148

In [7]:
df_damages.count()

24950

In [8]:
df_damages.show()

+--------+--------------------+
|CRASH_ID|    DAMAGED_PROPERTY|
+--------+--------------------+
|14768622|             MAILBOX|
|14768622|         YARD, GRASS|
|14838668|           GUARDRAIL|
|14838685|           ROAD SIGN|
|14838693|        2009 MAZDA 3|
|14838834|    CHAIN LINK FENCE|
|14838841|WOODED POLE ON SO...|
|14838842|CITY SIGN FOR TUR...|
|14838877|    FENCE-CHAIN LINK|
|14838977|LANDSCAPING AND M...|
|14839047|  APARTMENT BUILDING|
|14839047|         STREET SIGN|
|14839048|               HOUSE|
|14839314|          LIGHT POLE|
|14839330|MINOR DAMAGE TO W...|
|14839442|CITY OF SAN ANTON...|
|14839472|         METAL POLES|
|14839517|    WATER ATTENUATOR|
|14839519|        UTILITY POST|
|14839551|30 FEET OF GUARDRAIL|
+--------+--------------------+
only showing top 20 rows



In [9]:
df_endorse.count()

159818

In [10]:
df_endorse.show()

+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
|14838668|       1|              NONE|
|14838669|       1|              NONE|
|14838669|       2|              NONE|
|14838670|       1|              NONE|
|14838685|       1|        UNLICENSED|
|14838693|       1|              NONE|
|14838768|       1|OTHER/OUT OF STATE|
|14838768|       2|              NONE|
|14838834|       1|           UNKNOWN|
|14838841|       1|              NONE|
|14838842|       1|              NONE|
|14838842|       2|              NONE|
|14838866|       1|              NONE|
|14838877|       1|              NONE|
|14838972|       1|              NONE|
+--------+--------+------------------+
only showing top 20 rows



In [11]:
df_pp.count()

156954

In [12]:
df_pp.show()

+--------+--------+--------+------------+------------------+--------------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+--------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|   PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|       PRSN_REST_ID|   PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|    DRVR_LIC_TYPE_ID|DRVR_LIC_STATE_ID|   DRVR_LIC_CLS_ID|DRVR_ZIP

In [13]:
df_pp.columns

['CRASH_ID',
 'UNIT_NBR',
 'PRSN_NBR',
 'PRSN_TYPE_ID',
 'PRSN_OCCPNT_POS_ID',
 'PRSN_INJRY_SEV_ID',
 'PRSN_AGE',
 'PRSN_ETHNICITY_ID',
 'PRSN_GNDR_ID',
 'PRSN_EJCT_ID',
 'PRSN_REST_ID',
 'PRSN_AIRBAG_ID',
 'PRSN_HELMET_ID',
 'PRSN_SOL_FL',
 'PRSN_ALC_SPEC_TYPE_ID',
 'PRSN_ALC_RSLT_ID',
 'PRSN_BAC_TEST_RSLT',
 'PRSN_DRG_SPEC_TYPE_ID',
 'PRSN_DRG_RSLT_ID',
 'DRVR_DRG_CAT_1_ID',
 'PRSN_DEATH_TIME',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT',
 'DRVR_LIC_TYPE_ID',
 'DRVR_LIC_STATE_ID',
 'DRVR_LIC_CLS_ID',
 'DRVR_ZIP']

In [14]:
df_restrict.count()

159852

In [15]:
df_restrict.show()

+--------+--------+--------------------+
|CRASH_ID|UNIT_NBR| DRVR_LIC_RESTRIC_ID|
+--------+--------+--------------------+
|14768622|       1|                NONE|
|14838637|       1|  OTHER/OUT OF STATE|
|14838637|       2|                NONE|
|14838641|       1|WITH CORRECTIVE L...|
|14838641|       2|          UNLICENSED|
|14838668|       1|                NONE|
|14838669|       1|                NONE|
|14838669|       2|WITH CORRECTIVE L...|
|14838670|       1|                NONE|
|14838685|       1|          UNLICENSED|
|14838693|       1|                NONE|
|14838768|       1|  OTHER/OUT OF STATE|
|14838768|       2|WITH CORRECTIVE L...|
|14838834|       1|             UNKNOWN|
|14838841|       1|WITH CORRECTIVE L...|
|14838842|       1|                NONE|
|14838842|       2|                NONE|
|14838866|       1|                NONE|
|14838877|       1|                NONE|
|14838972|       1|                NONE|
+--------+--------+--------------------+
only showing top

In [16]:
df_Units.count()

173499

In [17]:
df_Units.show()

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+------------+---------------+--------------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID| VEH_MAKE_ID|     VEH_MOD_ID|    VEH_BODY_STYL_ID|EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|  VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|     VEH_TRANSP_NAME|     VEH_TRA

In [18]:
df_Units.columns

['CRASH_ID',
 'UNIT_NBR',
 'UNIT_DESC_ID',
 'VEH_PARKED_FL',
 'VEH_HNR_FL',
 'VEH_LIC_STATE_ID',
 'VIN',
 'VEH_MOD_YEAR',
 'VEH_COLOR_ID',
 'VEH_MAKE_ID',
 'VEH_MOD_ID',
 'VEH_BODY_STYL_ID',
 'EMER_RESPNDR_FL',
 'OWNR_ZIP',
 'FIN_RESP_PROOF_ID',
 'FIN_RESP_TYPE_ID',
 'VEH_DMAG_AREA_1_ID',
 'VEH_DMAG_SCL_1_ID',
 'FORCE_DIR_1_ID',
 'VEH_DMAG_AREA_2_ID',
 'VEH_DMAG_SCL_2_ID',
 'FORCE_DIR_2_ID',
 'VEH_INVENTORIED_FL',
 'VEH_TRANSP_NAME',
 'VEH_TRANSP_DEST',
 'CONTRIB_FACTR_1_ID',
 'CONTRIB_FACTR_2_ID',
 'CONTRIB_FACTR_P1_ID',
 'VEH_TRVL_DIR_ID',
 'FIRST_HARM_EVT_INV_ID',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT']

In [19]:
#Analytics_1
#Filter the data to consider only male fatalities
male_fatalities = df_pp.filter((col("PRSN_GNDR_ID") == "MALE") & (col("PRSN_INJRY_SEV_ID") == "KILLED"))

# For each CRASH_ID and count the number of male fatalities
male_fatalities_crash_count = male_fatalities.groupBy("CRASH_ID").agg({"CRASH_ID": "count"}).withColumnRenamed("count(CRASH_ID)", "male_fatalities_count")

# Filter crashes where the number of male fatalities is greater than 2
crashes_with_more_than_2_males_killed = male_fatalities_crash_count.filter(col("male_fatalities_count") > 2)

# Count the number of such crashes where male killed is more than 2
total_crashes = crashes_with_more_than_2_males_killed.count()

#result
print("Number of crashes with more than 2 males killed:", total_crashes)

Number of crashes with more than 2 males killed: 0


In [20]:
#Analytics_2
# Filter the data to consider only two-wheelers
#two_wheelers = df_Units.filter(col("VEH_BODY_STYL_ID") == "MOTORCYCLE")
two_wheelers = df_Units.filter(col("VEH_BODY_STYL_ID").contains("MOTORCYCLE"))

# Count the number of two-wheelers involved in crashes
num_two_wheelers_in_crashes = two_wheelers.count()

# Display the result
print("Number of two-wheelers involved in crashes:", num_two_wheelers_in_crashes)

Number of two-wheelers involved in crashes: 784


In [21]:
#Analytics_3
# Filter data for crashes where the driver died and airbags did not deploy
filtered_data_3 = df_pp.filter((F.col("DEATH_CNT") > 0) & (F.col("PRSN_AIRBAG_ID") == "NOT DEPLOYED"))

# Join with df_Units to get vehicle information
joined_data_3 = filtered_data_3.join(df_Units, "CRASH_ID", "inner")

# Group by vehicle make and count the occurrences
make_counts = joined_data_3.groupBy("VEH_MAKE_ID").count()

# Order by count in descending order
sorted_make_counts = make_counts.orderBy(F.desc("count"))

# Select the top 5 vehicle makes and filter out "NA"
top_5_makes = sorted_make_counts.filter(sorted_make_counts.VEH_MAKE_ID != "NA").limit(5).select("VEH_MAKE_ID")

# Display the result
top_5_makes.show()


+------------+
| VEH_MAKE_ID|
+------------+
|   CHEVROLET|
|        FORD|
|       DODGE|
|FREIGHTLINER|
|      NISSAN|
+------------+



In [22]:
#Analytics_4
# Filter relevant data for hit and run incidents
hit_and_run_df = df_charges.filter((F.col("CHARGE") == "HIT AND RUN") & (F.col("UNIT_NBR").isNotNull()))

# Join with the Units DataFrame to get information about the driver's license
joined_df_4 = hit_and_run_df.join(df_pp, on=['CRASH_ID', 'UNIT_NBR'], how='inner')

# Filter for vehicles with drivers having valid licenses
valid_license_df = joined_df_4.filter(F.col("DRVR_LIC_TYPE_ID").isNotNull())

# Count the number of vehicles
result_count = valid_license_df.select('CRASH_ID', 'UNIT_NBR').distinct().count()

print("Number of vehicles with drivers having valid licenses involved in hit and run:", result_count)

Number of vehicles with drivers having valid licenses involved in hit and run: 25


In [23]:
#Analytics_5
# Join relevant DataFrames (df_pp, df_Units, etc.) on the common key (CRASH_ID)
joined_df_5 = df_pp.join(df_Units, on=['CRASH_ID', 'UNIT_NBR'], how='inner')

# Filter for accidents where females are not involved
accidents_without_females = joined_df_5.filter(F.col("PRSN_GNDR_ID") != 'Female')

# Group by state and count the number of accidents
state_accident_counts = accidents_without_females.groupBy("VEH_LIC_STATE_ID").count()

# Find the state with the highest number of accidents
max_accident_state = state_accident_counts.orderBy(F.desc("count")).first()["VEH_LIC_STATE_ID"]

print("State with the highest number of accidents without females:", max_accident_state)

State with the highest number of accidents without females: TX


In [24]:
#Analytics_6
# Group by VEH_MAKE_ID and calculate the total injury count
injury_counts = df_Units.groupBy("VEH_MAKE_ID").agg(
    F.sum("TOT_INJRY_CNT").alias("TOTAL_INJURY_COUNT"),
    F.sum("DEATH_CNT").alias("DEATH_COUNT")
)

# Calculate the total injury count including death
injury_counts = injury_counts.withColumn(
    "TOTAL_INJURY_COUNT_INCLUDING_DEATH",
    F.col("TOTAL_INJURY_COUNT") + F.col("DEATH_COUNT")
)

# Rank the VEH_MAKE_IDs based on the total injury count including death
window_spec = Window.orderBy(F.desc("TOTAL_INJURY_COUNT_INCLUDING_DEATH"))

injury_ranking = injury_counts.withColumn(
    "rank",
    F.rank().over(window_spec)
)

# Filter for the top 3rd to 5th VEH_MAKE_IDs
result = injury_ranking.filter((F.col("rank") >= 3) & (F.col("rank") <= 5)).select("VEH_MAKE_ID")

# Show the result
result.show()


+-----------+
|VEH_MAKE_ID|
+-----------+
|     TOYOTA|
|      DODGE|
|     NISSAN|
+-----------+



In [25]:
#Analytics_7
# Select relevant columns
df_pp_selected = df_pp.select(
    "CRASH_ID", "PRSN_ETHNICITY_ID"
)

df_Units_selected = df_Units.select(
    "CRASH_ID", "VEH_BODY_STYL_ID"
)

# Join df_Units and df_pp on CRASH_ID
joined_df_7 = df_Units_selected.join(df_pp_selected, "CRASH_ID")

# Filter out undesired values in VEH_BODY_STYL_ID
filtered_joined_df_7 = joined_df_7.filter(
    ~joined_df_7.VEH_BODY_STYL_ID.isin("NA", "NOT REPORTED", "OTHER", "UNKNOWN")
)

# Group by VEH_BODY_STYL_ID and PRSN_ETHNICITY_ID, calculate the count
grouped_df_7 = filtered_joined_df_7.groupBy("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").agg(
    F.count("PRSN_ETHNICITY_ID").alias("ethnicity_count")
)

# Rank the ethnic groups within each body style based on the count
window_spec = Window.partitionBy("VEH_BODY_STYL_ID").orderBy(F.desc("ethnicity_count"))

ranked_df = grouped_df_7.withColumn(
    "rank",
    F.rank().over(window_spec)
)

# Select the top ethnic group for each body style
result = ranked_df.filter(F.col("rank") == 1).select("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID")

# Show the result
result.show(truncate=False)


+---------------------------------+-----------------+
|VEH_BODY_STYL_ID                 |PRSN_ETHNICITY_ID|
+---------------------------------+-----------------+
|AMBULANCE                        |WHITE            |
|BUS                              |HISPANIC         |
|FARM EQUIPMENT                   |WHITE            |
|FIRE TRUCK                       |WHITE            |
|MOTORCYCLE                       |WHITE            |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|WHITE            |
|OTHER  (EXPLAIN IN NARRATIVE)    |WHITE            |
|PASSENGER CAR, 2-DOOR            |WHITE            |
|PASSENGER CAR, 4-DOOR            |WHITE            |
|PICKUP                           |WHITE            |
|POLICE CAR/TRUCK                 |WHITE            |
|POLICE MOTORCYCLE                |HISPANIC         |
|SPORT UTILITY VEHICLE            |WHITE            |
|TRUCK                            |WHITE            |
|TRUCK TRACTOR                    |WHITE            |
|VAN                        

In [26]:
#Analytics_8
# Select relevant columns
df_selected_8 = df_pp.select("CRASH_ID", "DRVR_ZIP", "PRSN_ALC_RSLT_ID")

# Filter rows where alcohol is a contributing factor
df_filtered_8 = df_selected_8.filter(F.col("PRSN_ALC_RSLT_ID") == "Positive")

# Filter out rows with null values in DRVR_ZIP
df_filtered_8 = df_filtered_8.filter(df_filtered_8.DRVR_ZIP.isNotNull())

# Group by driver's zip code and count the number of crashes
zip_code_counts = df_filtered_8.groupBy("DRVR_ZIP").agg(
    F.count("CRASH_ID").alias("crash_count")
)

# Sort the result in descending order and select the top 5 zip codes
top_zip_codes = zip_code_counts.sort(F.desc("crash_count")).limit(5).select("DRVR_ZIP")

# Show the result
top_zip_codes.show(truncate=False)


+--------+
|DRVR_ZIP|
+--------+
|78521   |
|76010   |
|79936   |
|79938   |
|79907   |
+--------+



In [27]:
#Analytics_9
# Select relevant columns from df_damages
df_damages_selected = df_damages.select("CRASH_ID", "DAMAGED_PROPERTY")

# Select relevant columns from df_Units
df_units_selected = df_Units.select("CRASH_ID", "VEH_DMAG_SCL_1_ID", "FIN_RESP_PROOF_ID")

# Join dataframes on CRASH_ID
joined_df_9 = df_damages_selected.join(df_units_selected, "CRASH_ID", "inner")

# Filter based on the conditions
filtered_df_9 = joined_df_9.filter(
    (F.col("DAMAGED_PROPERTY").isNull()) &  # No damaged property observed
    (F.col("VEH_DMAG_SCL_1_ID").isin('DAMAGED 5', 'DAMAGED 7 HIGHEST', 'DAMAGED 6')) &  # Damage level above 4
    (F.col("FIN_RESP_PROOF_ID").like('%INSURANCE%'))  # Car has insurance
)

# Count distinct Crash IDs
distinct_crash_count = filtered_df_9.select("CRASH_ID").distinct().count()

# Show the result
print(f"Count of Distinct Crash IDs where there is no damage of property: {distinct_crash_count}")

Count of Distinct Crash IDs where there is no damage of property: 0


In [28]:
#To get TOP_STATES
#Filter out rows with "NA", "Other", or "Unknown" in "DRVR_LIC_STATE_ID"
filtered_df_pp = df_pp.filter((df_pp.DRVR_LIC_STATE_ID.isNotNull()) & 
                              (df_pp.DRVR_LIC_STATE_ID != "NA") & 
                              (df_pp.DRVR_LIC_STATE_ID != "Other") & 
                              (df_pp.DRVR_LIC_STATE_ID != "Unknown"))

# Group by "DRVR_LIC_STATE_ID", count occurrences, order by count in descending order, limit to top 25
state_counts = (
    filtered_df_pp
    .groupBy("DRVR_LIC_STATE_ID")
    .agg(F.count("*").alias("count"))
    .orderBy(F.desc("count"))
    .limit(25)
)

# Set the configuration parameter for eager evaluation to display more rows
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# Display the result
state_counts.show(25,truncate=False)

+-----------------+------+
|DRVR_LIC_STATE_ID|count |
+-----------------+------+
|Texas            |136335|
|Mexico           |1007  |
|Louisiana        |752   |
|New Mexico       |699   |
|California       |596   |
|Florida          |544   |
|Oklahoma         |512   |
|Arkansas         |344   |
|Arizona          |256   |
|Georgia          |247   |
|Colorado         |203   |
|Illinois         |196   |
|Tennessee        |182   |
|Missouri         |181   |
|Mississippi      |180   |
|North Carolina   |161   |
|Kansas           |141   |
|Alabama          |135   |
|Michigan         |132   |
|Washington       |127   |
|Ohio             |119   |
|New York         |114   |
|Virginia         |108   |
|South Carolina   |101   |
|Pennsylvania     |88    |
+-----------------+------+



In [29]:
#TO GET TOP_COLORS
filtered_df_Units = df_Units.filter((df_Units.VEH_COLOR_ID.isNotNull()) & 
                              (df_Units.VEH_COLOR_ID != "NA") )

# Group by "VEH_COLOR_ID", count occurrences, order by count in descending order, limit to top 10
color_counts = (
    filtered_df_Units
    .groupBy("VEH_COLOR_ID")
    .agg(F.count("*").alias("count"))
    .orderBy(F.desc("count"))
    .limit(10)
)

# Select only the "VEH_COLOR_ID" column from the result
top_colors = color_counts.select("VEH_COLOR_ID")#.rdd.flatMap(lambda x: x).collect()

# Display the result
#top_colors.show(truncate=False)
top_colors.show()

+------------+
|VEH_COLOR_ID|
+------------+
|         WHI|
|         BLK|
|         SIL|
|         GRY|
|         BLU|
|         RED|
|         GRN|
|         MAR|
|         TAN|
|         GLD|
+------------+



In [30]:
#Analytics_10
# Filter relevant columns for each dataframe
df_charges_filtered_10 = df_charges.select("CRASH_ID", "UNIT_NBR", "PRSN_NBR", "CHARGE")
df_pp_filtered_10 = df_pp.select("CRASH_ID", "UNIT_NBR", "DRVR_LIC_STATE_ID")
df_Units_filtered_10 = df_Units.select("CRASH_ID", "VEH_MAKE_ID", "VEH_COLOR_ID", "VEH_LIC_STATE_ID")

# Define your top_colors and top_states manually as lists
top_colors = ["WHI", "BLK", "SIL", "GRY", "BLU", "RED", "GRN", "MAR", "TAN", "GLD"]
top_states = ["Texas", "Mexico", "Louisiana", "New Mexico", "California", "Florida", "Oklahoma", "Arkansas", "Arizona", "Georgia", "Colorado", "Illinois", "Tennessee", "Missouri", "Mississippi", "North Carolina", "Kansas", "Alabama", "Michigan", "Washington", "Ohio", "New York", "Virginia", "South Carolina", "Pennsylvania"]

# Filter data based on conditions using provided lists
filtered_data_10 = df_charges_filtered_10.join(df_pp_filtered_10, on="CRASH_ID", how="inner") \
    .join(df_Units_filtered_10, on="CRASH_ID", how="inner") \
    .filter((F.col("CHARGE").contains("SPEED")) &
            (F.col("DRVR_LIC_STATE_ID").isin(top_states)) &
            (F.col("VEH_COLOR_ID").isin(top_colors)))

# Count the occurrences of each vehicle make
top_vehicle_makes = filtered_data_10.groupBy("VEH_MAKE_ID").count().orderBy(F.desc("count")).limit(5).select("VEH_MAKE_ID")

# Show the result
top_vehicle_makes.show(truncate=False)


+-----------+
|VEH_MAKE_ID|
+-----------+
|FORD       |
|CHEVROLET  |
|TOYOTA     |
|DODGE      |
|NISSAN     |
+-----------+

